In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_capillary',
    'capillary_daily_campaign',
    proc_date,
    'temp_capillary_daily_campaign'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_capillary.capillary_daily_campaign
(
    campaign_id STRING,
    campaign_name STRING,
    start_date STRING,
    end_date STRING,
    user_id STRING,
    segment STRING,
    delivery_status STRING,
    proc_date DATE,
    file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_capillary.capillary_daily_campaign
    WHERE proc_date = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_capillary.capillary_daily_campaign
SELECT
campaign_campaign_id as campaign_id,
campaign_name as campaign_name,
campaign_campaign_start_date as start_date,
campaign_campaign_end_date as end_date,
user_user_id as user_id,
group_type as segment,
campaign_delivery_status as delivery_status,
proc_date,
file_creation_ts TIMESTAMP
FROM temp_capillary_daily_campaign;
""")